In [1]:
import random
import numpy as np
import numpy.ma as ma
import diff_classifier.msd as msd

In [21]:
random.seed(a=1)
mask = np.concatenate((np.zeros((50, 5)), np.ones((50, 5))), axis=1)
x = ma.array(np.random.rand(50, 10), mask=mask)
random.seed(a=2)
y = ma.array(np.random.rand(50, 10), mask=mask)
parts = x.shape[0]
frames = x.shape[1]

xfilled = ma.copy(x)
yfilled = ma.copy(y)
for shift in range(1,frames):
    x_shifted=np.roll(x,shift=shift,axis=1)
    idx=~x_shifted.mask * xfilled.mask
    xfilled[idx]=x_shifted[idx]
    y_shifted=np.roll(y,shift=shift,axis=1)
    idy=~y_shifted.mask * yfilled.mask
    yfilled[idy]=y_shifted[idy]
xfilled = xfilled.data
yfilled = yfilled.data

msds = ma.array(np.linspace(0, 10, 10) + x, mask=mask)
delt = ma.array(np.linspace(0, 10, 10)+ 0*x, mask=mask)

In [ ]:
numerator = np.square(msd.nth_diff(xfilled, n=frames-1, axis=1))+np.square(msd.nth_diff(yfilled, n=frames-1, axis=1))
denominator = np.reshape(np.sum(np.square(msd.nth_diff(x, axis=1)), axis=1), (parts, 1)) + np.reshape(np.sum(np.square(msd.nth_diff(y, axis=1)), axis=1), (parts, 1))
efficiency = numerator/denominator.data

In [ ]:
numerator = np.abs(msd.nth_diff(xfilled, n=frames-1, axis=1))+np.abs(msd.nth_diff(yfilled, n=frames-1, axis=1))
denominator = np.reshape(np.sum(np.abs(msd.nth_diff(x, axis=1)), axis=1), (parts, 1)) + np.reshape(np.sum(np.abs(msd.nth_diff(y, axis=1)), axis=1), (parts, 1))
straightness = numerator/denominator.data

In [ ]:
N = x.count(axis=1)
L = np.sum(np.sqrt(np.square(msd.nth_diff(x, axis=1))), axis=1)

diff = np.zeros((parts, frames, frames-1))
for i in range(frames-1):
    diff[:, 0:frames-i-1, i] = np.square(msd.nth_diff(x, n=i+1, axis=1))
d = np.max(np.max(diff, axis=2), axis=1)

fractal_dim = np.log(N)/np.log(N*d/L).data

In [ ]:
fractal_dim

In [65]:
from scipy.optimize import curve_fit
def anom_msd(delt, D, alpha):
    msd = 4*D*(delt**alpha)
    return np.array(msd).ravel()

In [47]:
anom_msd(delt[1:5, :].data, 0.5, 1)

array([[  0.        ,   2.22222222,   4.44444444,   6.66666667,
          8.88888889,  11.11111111,  13.33333333,  15.55555556,
         17.77777778,  20.        ],
       [  0.        ,   2.22222222,   4.44444444,   6.66666667,
          8.88888889,  11.11111111,  13.33333333,  15.55555556,
         17.77777778,  20.        ],
       [  0.        ,   2.22222222,   4.44444444,   6.66666667,
          8.88888889,  11.11111111,  13.33333333,  15.55555556,
         17.77777778,  20.        ],
       [  0.        ,   2.22222222,   4.44444444,   6.66666667,
          8.88888889,  11.11111111,  13.33333333,  15.55555556,
         17.77777778,  20.        ]])

In [63]:
ma.reshape(msds.ravel(), (50, 10))

masked_array(data =
 [[0.8263403340576346 1.7599891840604496 2.3780309607222447
  4.234786106293765 4.5102212622552695 -- -- -- -- --]
 [0.6927464717087011 1.2073894629876705 2.409516559790498
  3.8331052073738396 4.836652513397638 -- -- -- -- --]
 [0.019241494747452625 1.4018807707686802 2.3525753764363126
  3.9761697316979685 5.026133148336873 -- -- -- -- --]
 [0.8754078372140571 2.0028210721776065 3.1232248374302967
  3.6745029137719474 5.084875218594816 -- -- -- -- --]
 [0.678171408119193 1.6260840104639918 3.170718788909899 4.002728973727632
  4.675797508731729 -- -- -- -- --]
 [0.8970724250632548 1.6799373136682223 2.8990107705292325
  4.046274191702705 4.703593404464024 -- -- -- -- --]
 [0.6332114572043579 1.8815735859713536 2.616988679790526
  3.4848908738704187 5.14233583123845 -- -- -- -- --]
 [0.7864905085918827 1.7113956648895563 3.2172301206054916
  4.226050610024641 4.815343919792272 -- -- -- -- --]
 [0.7815538527225653 2.0697525206540095 2.625932820008103
  4.17461393849

In [67]:
popt, pcov = curve_fit(anom_msd, delt.data, msds.data.ravel())

In [68]:
popt

array([ 0.32421309,  0.87825658])